In [14]:
from abc import ABC, abstractmethod
import re


class Equation(ABC):
    # Atributos de clase que deben ser definidos por las subclases
    degree: int  # Grado de la ecuación
    type: str    # Tipo de ecuación
  
    def __init__(self, *args):
        # Constructor que valida los argumentos recibidos
        if (self.degree + 1) != len(args):
            # Verifica que el número de coeficientes coincida con el grado de la ecuación
            raise TypeError(
                f"'Equation' object takes {self.degree + 1} positional arguments but {len(args)} were given"
            )
        if any(not isinstance(arg, (int, float)) for arg in args):
            # Verifica que todos los coeficientes sean números (enteros o flotantes)
            raise TypeError("Coefficients must be of type 'int' or 'float'")
        if args[0] == 0:
            # Verifica que el coeficiente de mayor grado no sea cero
            raise ValueError("Highest degree coefficient must be different from zero")
        # Crea un diccionario de coeficientes donde la clave es el grado del término
        self.coefficients = {(len(args) - n - 1): arg for n, arg in enumerate(args)}

    def __init_subclass__(cls):
        # Método que se ejecuta cuando se crea una subclase de Equation
        if not hasattr(cls, "degree"):
            # Verifica que la subclase tenga definido el atributo 'degree'
            raise AttributeError(
                f"Cannot create '{cls.__name__}' class: missing required attribute 'degree'"
            )
        if not hasattr(cls, "type"):
            # Verifica que la subclase tenga definido el atributo 'type'
            raise AttributeError(
                f"Cannot create '{cls.__name__}' class: missing required attribute 'type'"
            )

    def __str__(self):
         # Función auxiliar para convertir números a superíndices Unicode
        def to_superscript(n):
            # Diccionario de dígitos superíndices Unicode
            superscripts = {
                '0': '⁰', '1': '¹', '2': '²', '3': '³', '4': '⁴',
                '5': '⁵', '6': '⁶', '7': '⁷', '8': '⁸', '9': '⁹'
            }
            # Convierte cada dígito del número a su equivalente superíndice
            return ''.join(superscripts[digit] for digit in str(n))
        
        # Método que devuelve una representación en string de la ecuación
        terms = []
        for n, coefficient in self.coefficients.items():
            if not coefficient:  # Salta los coeficientes que sean cero
                continue
            if n == 0:  # Caso para el término constante
                terms.append(f'{coefficient:+}')
            elif n == 1:  # Caso para el término lineal
                terms.append(f'{coefficient:+}x')
            else:  # Caso para términos con exponente > 1
                 # Usa la función to_superscript para convertir el exponente
                terms.append(f"{coefficient:+}x{to_superscript(n)}")
    
        # Une todos los términos con espacios y añade "= 0" al final
        equation_string = ' '.join(terms) + ' = 0'
        # Reemplaza "1x" por "x" usando expresiones regulares y elimina el signo "+" al inicio si existe
        return re.sub(r"(?<!\d)1(?=x)", "", equation_string.strip("+"))        

    @abstractmethod
    def solve(self):
        # Método abstracto que debe ser implementado por las subclases
        # para resolver la ecuación
        pass
        
    @abstractmethod
    def analyze(self):
        # Método abstracto que debe ser implementado por las subclases
        # para analizar propiedades de la ecuación
        pass


class LinearEquation(Equation):
    # Subclase para ecuaciones lineales (ax + b = 0)
    degree = 1  # Grado de la ecuación lineal
    type = 'Linear Equation'  # Tipo de ecuación
    
    def solve(self):
        # Resuelve la ecuación lineal
        a, b = self.coefficients.values()  # Desempaqueta los coeficientes
        x = -b / a  # Calcula la solución (para ax + b = 0, x = -b/a)
        return [x]  # Devuelve la solución como una lista con un solo elemento

    def analyze(self):
        # Analiza propiedades de la ecuación lineal
        slope, intercept = self.coefficients.values()  # Desempaqueta los coeficientes
        # Retorna un diccionario con la pendiente y el intercepto
        return {'slope': slope, 'intercept': intercept}


class QuadraticEquation(Equation):
    # Subclase para ecuaciones cuadráticas (ax² + bx + c = 0)
    degree = 2  # Grado de la ecuación cuadrática
    type = 'Quadratic Equation'  # Tipo de ecuación

    def __init__(self, *args):
        # Constructor específico para ecuaciones cuadráticas
        super().__init__(*args)  # Llama al constructor de la clase padre
        a, b, c = self.coefficients.values()  # Extrae los coeficientes a, b, c
        self.delta = b**2 - 4 * a * c  # Calcula el discriminante Δ = b² - 4ac

    def solve(self):
        # Resuelve la ecuación cuadrática
        if self.delta < 0:  # Si el discriminante es negativo, no hay soluciones reales
            return []
        a, b, _ = self.coefficients.values()  # Extrae los coeficientes a, b
        # Calcula las raíces usando la fórmula cuadrática
        x1 = (-b + (self.delta) ** 0.5) / (2 * a)
        x2 = (-b - (self.delta) ** 0.5) / (2 * a)
        if self.delta == 0:  # Si el discriminante es cero, hay una sola raíz (doble)
            return [x1]
        return [x1, x2]  # Devuelve las raíces como una lista

    def analyze(self):
        # Analiza propiedades de la ecuación cuadrática
        a, b, c = self.coefficients.values()  # Extrae los coeficientes a, b, c
        x = -b / (2 * a)  # Coordenada x del vértice
        y = a * x**2 + b * x + c  # Coordenada y del vértice
        # Determina la concavidad y si el vértice es mínimo o máximo
        if a > 0:
            concavity = 'upwards'  # Concavidad hacia arriba
            min_max = 'min'  # El vértice es un mínimo
        else:
            concavity = 'downwards'  # Concavidad hacia abajo
            min_max = 'max'  # El vértice es un máximo
        # Retorna un diccionario con las propiedades analizadas
        return {'x': x, 'y': y, 'min_max': min_max, 'concavity': concavity}


def solver(equation):
    # Función que resuelve y formatea la salida para una ecuación
    if not isinstance(equation, Equation):
        # Verifica que el argumento sea una instancia de Equation
        raise TypeError("Argument must be an Equation object")

    # Crea una cadena con el tipo de ecuación centrado en un ancho de 24 caracteres, rellenado con guiones
    output_string = f'\n{equation.type:-^24}'
    # Añade la representación de la ecuación como cadena, centrada en un ancho de 24 caracteres
    output_string += f'\n\n{equation!s:^24}\n\n'
    # Añade el título "Solutions" centrado en un ancho de 24 caracteres, rellenado con guiones
    output_string += f'{"Solutions":-^24}\n\n'
    
    # Llama al método solve() de la ecuación y guarda el resultado
    results = equation.solve()
    
    # Usa pattern matching para verificar la longitud de los resultados
    match results:
        case []:
            # No hay raíces reales
            result_list = ['No real roots']
        case [x]:
            # Una sola raíz
            result_list = [f'x = {x:+.3f}']
        case [x1, x2]:
            # Dos raíces
            result_list = [f'x1 = {x1:+.3f}', f'x2 = {x2:+.3f}']
    
    # Añade cada resultado a la cadena de salida, centrado en un ancho de 24 caracteres
    for result in result_list:
        output_string += f'{result:^24}\n'
    
    # Añade el título "Details" centrado en un ancho de 24 caracteres, rellenado con guiones
    output_string += f'\n{"Details":-^24}\n\n'
    
    # Obtiene los detalles del análisis de la ecuación
    details = equation.analyze()
    
    # Usa pattern matching para formatear los detalles según el tipo de ecuación
    match details:
        case {'slope': slope, 'intercept': intercept}:
            # Para ecuaciones lineales, formatea la pendiente y el intercepto
            details_list = [f'slope = {slope:>16.3f}', f'y-intercept = {intercept:>10.3f}']
        case {'x': x, 'y': y, 'min_max': min_max, 'concavity': concavity}:
            # Para ecuaciones cuadráticas, formatea la concavidad y el punto mínimo/máximo
            coord = f'({x:.3f}, {y:.3f})'
            details_list = [f'concavity = {concavity:>12}', f'{min_max} = {coord:>18}']
    
    # Añade cada detalle a la cadena de salida
    for detail in details_list:
        output_string += f'{detail}\n'
    
    # Devuelve la cadena de salida completa
    return output_string

# Crea una instancia de ecuación lineal con coeficientes 2 y 3 (2x + 3 = 0)
lin_eq = LinearEquation(2, 3)

# Crea una instancia de ecuación cuadrática con coeficientes 1, 2 y 1 (x² + 2x + 1 = 0)
quadr_eq = QuadraticEquation(-3, 2, 1)

# Imprime la solución formateada de la ecuación cuadrática
print(solver(quadr_eq))



---Quadratic Equation---

    -3x² +2x +1 = 0     

-------Solutions--------

      x1 = -0.333       
      x2 = +1.000       

--------Details---------

concavity =    downwards
max =     (0.333, 1.333)

